# ginzaの使い方

## 基本的な使い方

モデルを指定して解析器を読み込む．

In [1]:
import spacy

nlp = spacy.load("ja_ginza")

その解析器を用いて，文章を単語に分割することができる．

In [2]:
text = "昨日の天気は雨でした。晴れてほしかった。"
doc = nlp(text)

for token in doc:
    print(token)

昨日
の
天気
は
雨
でし
た
。
晴れ
て
ほしかっ
た
。


分割された単語には，出現形(text)，原形(lemma_)，品詞(tag_)が属性として付与される[<sup>1</sup>](#id1)．

In [3]:
for token in doc:
    print(f"{token.text}\t{token.lemma_}\t{token.pos_}\t{token.tag_}")

昨日	昨日	NOUN	名詞-普通名詞-副詞可能
の	の	ADP	助詞-格助詞
天気	天気	NOUN	名詞-普通名詞-一般
は	は	ADP	助詞-係助詞
雨	雨	NOUN	名詞-普通名詞-一般
でし	です	AUX	助動詞
た	た	AUX	助動詞
。	。	PUNCT	補助記号-句点
晴れ	晴れる	VERB	動詞-一般
て	て	SCONJ	助詞-接続助詞
ほしかっ	ほしい	AUX	形容詞-非自立可能
た	た	AUX	助動詞
。	。	PUNCT	補助記号-句点


pandasのDataFrameに解析結果を格納する方法はこのようになる．

In [4]:
import pandas as pd

df = pd.DataFrame({
    'text': token.text,
    'lemma_': token.lemma_,
    'pos_': token.pos_,
    'tag_': token.tag_
    } for token in doc)

display(df)

,text,lemma_,pos_,tag_
0,昨日,昨日,NOUN,名詞-普通名詞-副詞可能
1,の,の,ADP,助詞-格助詞
2,天気,天気,NOUN,名詞-普通名詞-一般
3,は,は,ADP,助詞-係助詞
4,雨,雨,NOUN,名詞-普通名詞-一般
5,でし,です,AUX,助動詞
6,た,た,AUX,助動詞
7,。,。,PUNCT,補助記号-句点
8,晴れ,晴れる,VERB,動詞-一般
9,て,て,SCONJ,助詞-接続助詞


さきほど用意した小説を単語に分割してみよう．

In [5]:
input_fn = 'text/kageotoko.corpus.txt'
output_fn = 'text/kageotoko.wakati.txt'

with open(input_fn, 'r') as fin, open(output_fn, 'w') as fout:
    for line in fin:
        tokens = [token.text for token in nlp(line.rstrip())]
        fout.write(' '.join(tokens)+'\n')

!head text/kageotoko.wakati.txt

断末魔 の 雄 獅子

　 三十二 、 三 歳 に 見える やせ型 の 男 が 、 張 ホテル の 玄関 を はいっ て 、 カウンター の うしろ の 支配人 室 へ 踏みこん で いっ た 。
　 ずんぐり と 背 が 低く て 丸々 と 太っ た ちょびひげ の 支配人 が デスク に 向かっ て 帳簿 を いじくっ て い た 。 そば の 灰 ざら に のせ た 半分 ほど に なっ た 葉巻き から 、 細い 紫色 の 煙 が ほとんど まっすぐ に 立ちのぼっ て い た 。 ハバナ の かおり が 何 か 猥 ※ な 感じ で 漂っ て い た 。
「 来 て いる ね ？ 」
　 やせ型 の 男 が ニヤッ と 笑っ て たずね た 。
「 うん 、 来 て いる 。 もう 始まっ て いる ころ だ よ 」
「 じゃあ 、 あの へや へ 行く よ 」
「 いい とも 、 見つかり っこ は ない が 、 せいぜい 用心 し て ね 」
　 やせ型 の 男 は ネズミ色 の セビロ を 着 て 、 ネズミ色 の ワイシャツ 、 ネズミ色 の ネクタイ 、 くつ まで ネズミ色 の もの を はい て い た 。 どんな 背景 の 前 で も 最も 目だた ない 服装 で あっ た 。 かれ は まったく 足音 を たて ない で 階段 を 駆け上がり 、 二 階 の ずっと 奥まっ た 一室 の ドア を そっと ひらい て 、 中 に すべりこむ と 、 電灯 も つけ ず 、 一方 の 壁 に ある 押し入れ の 戸 を 用意 の カギ で ひらき 、 その 中 へ 身 を 隠し た 。


単語の出現頻度を数えてみよう．活用する語の活用形を別々に数えるのではなく，原形で集計する．

In [6]:
from collections import Counter

include_pos = ('NOUN', 'VERB', 'ADJ')
stopwords = ('する', 'ある', 'ない', 'いう', 'もの', 'こと', 'よう', 'なる', 'ほう')

SudachiPyのバージョンが0.6以上の場合，分析できるテキストの長さに限界があるため，100行ずつまとめて処理する

In [13]:
def chunk(lst, n):
    """
    リストをn個ずつのサブリストに分割する。
    """
    return [lst[i:i+n] for i in range(0, len(lst), n)]


with open(input_fn, 'r') as f:
    lines = [l.replace("\n","") for l in f.readlines()]

lst_texts = chunk(lines,100)
lst_tokens = []
for text in lst_texts:
    doc = nlp(" ".join(text))
    lst_tokens.extend([
        token.lemma_ for token in doc
        if token.pos_ in include_pos and token.lemma_ not in stopwords
    ])
counter = Counter(lst_tokens)

print('count word')
for word, count in counter.most_common(10):
     print(f'{count:>5} {word}')

count word
 1262 いる
  225 お
  212 中
  206 人
  199 男
  193 くる
  175 顔
  157 いく
  156 ふたり
  143 つ


## 注釈
<sup>1</sup> <span id=id1>pos_属性は，Universal Dependencyの品詞体系．</span>